# Caso: Análisis de Fuga de una Empresa de Telecomunicaciones

## Cargar archivos csv
Utilizamos un Call Detail Record de ejemplo
##### Clientes llamantes: 199
##### Clientes llamados: 400
##### Horizonte de tiempo: De Nov-2012 a Abr-2013 (6 meses)

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import networkx as nx
import time

In [ ]:
ll = pd.read_csv('../data/llamadas2.csv')

Revisamos la información del DataFrame

In [ ]:
ll.info()

In [ ]:
ll.head()

Transformamos el DataFrame en un Grafo dirigido con pesos (cantidad de llamadas y duración)

In [ ]:
G = nx.from_pandas_edgelist(ll, source="fromuserid", target="touserid", edge_attr=["numbercalls","secondscalls"], create_using=nx.DiGraph())
print(nx.info(G))

Cargamos la lista de flags de baja

In [ ]:
fl = pd.read_csv('../data/flagc.csv',index_col="fromuserid")

In [ ]:
fl.head()

In [ ]:
fl.info()

Agregamos a cada nodo el atributo de fuga.

In [ ]:
nx.set_node_attributes(G, dict(fl["flagc"]), 'churn')
nx.set_node_attributes(G, dict(fl["baj1"]), 'baj1')
nx.set_node_attributes(G, dict(fl["baj2"]), 'baj2')
nx.set_node_attributes(G, dict(fl["baj3"]), 'baj3')

Visualización del grafo representando los estados de cada nodo a Abril 2013 ( 0 = No fuga; 1 = Fuga en Ene-Feb y 2 = Fuga en Mar-Abr)

In [ ]:
plt.figure(figsize=(80,45)) 
node_colors = ['b' if G.nodes[v]['churn'] == 1
               else 'r' if G.nodes[v]['churn'] == 2
               else 'g' for v in G]
nx.draw_networkx(G, width=0.1, label=True, node_color=node_colors)
plt.show()

Identificamos algunos nodos que podrían estar relacionados con los churners

In [ ]:
list(G.successors(350007))

In [ ]:
list(G.predecessors(350007))

## Método 1: Extraer un Subgrafo tipo Árbol con bfs_tree (Manual)

In [ ]:
S = nx.bfs_tree(G,350007,depth_limit=2)
plt.figure(figsize=(64,36))
nx.draw_networkx(S, width=0.1)
plt.show()

Extraemos un subgrafo para analizar la incidencia de la Fuga en Ene-Feb

In [ ]:
SG = nx.subgraph(G, [350002,350006,350007,350009,350008,350010,350011,350012,350014,350019,350020,350022,350023,350026,350029,350032,350033,350050,350051,350060,350107,350186])
plt.figure(figsize=(16,9))
spr = nx.spring_layout(SG, k=0.3)
node_colors = ['b' if SG.nodes[v]['churn'] == 1
               else 'g' for v in SG]
nx.draw_networkx(SG, width=0.1, label=True, pos=spr, node_color=node_colors)
plt.show()

Ahora visualizamos el efecto de propagación en Mar-Abr

In [ ]:
plt.figure(figsize=(16,9))
node_colors = ['b' if SG.nodes[v]['churn'] == 1
               else 'r' if SG.nodes[v]['churn'] == 2
               else 'g' for v in SG]
nx.draw_networkx(SG, width=0.1, label=True, pos=spr, node_color=node_colors)
plt.show()

También podemos visualizar el efecto de propagación en May-Jun

In [ ]:
plt.figure(figsize=(16,9))
node_colors = ['b' if SG.nodes[v]['churn'] == 1
               else 'r' if SG.nodes[v]['churn'] == 2
               else 'y' if SG.nodes[v]['churn'] == 3
               else 'g' for v in SG]
nx.draw_networkx(SG, width=0.1, label=True, pos=spr, node_color=node_colors)
plt.show()

## Método 2: Extraer un Subgrafo tipo Ego (Considerando los dos primeros clientes que fugaron)

In [ ]:
E1=nx.ego_graph(G.to_undirected(),350001)
E2=nx.ego_graph(G.to_undirected(),350010)
E=nx.compose(E1,E2)
plt.figure(figsize=(16,9))
nx.draw_networkx(E, width=0.1)
plt.show()

Observamos la incidencia de la Fuga en Ene-Feb

In [ ]:
plt.figure(figsize=(32,18))
spr = nx.spring_layout(E, k=0.3)
node_colors = ['b' if E.nodes[v]['churn'] == 1
               else 'g' for v in E]
nx.draw_networkx(E, width=0.1, label=True, pos=spr, node_color=node_colors)
plt.show()

Ahora visualizamos el efecto de propagación en Mar-Abr

In [ ]:
plt.figure(figsize=(32,18))
node_colors = ['b' if E.nodes[v]['churn'] == 1
               else 'r' if E.nodes[v]['churn'] == 2
               else 'g' for v in E]
nx.draw_networkx(E, width=0.1, label=True, pos=spr, node_color=node_colors)
plt.show()

También podemos visualizar el efecto de propagación en May-Jun

In [ ]:
plt.figure(figsize=(32,18))
node_colors = ['b' if E.nodes[v]['churn'] == 1
               else 'r' if E.nodes[v]['churn'] == 2
               else 'y' if E.nodes[v]['churn'] == 3
               else 'g' for v in E]
nx.draw_networkx(E, width=0.1, label=True, pos=spr, node_color=node_colors)
plt.show()

## Análisis de Contagio del Churn
Generamos la función de cálculo de influencia promedio de los churners acumulados hasta Abril sobre los otros clientes

In [ ]:
def calculate_avg_score(G, node_id):
    score_sum = 0
    n_neighbors = 0
    for neighbor_id in G[node_id]:
        score_sum += G.nodes[neighbor_id]['baj2']
        n_neighbors += 1
    if n_neighbors==0:
        return 0
    else:
        return score_sum / n_neighbors

In [ ]:
calculate_avg_score(E, 350002)

In [ ]:
calculate_avg_score(E, 350017)

In [ ]:
calculate_avg_score(E, 350023)

In [ ]:
calculate_avg_score(E, 350109)

Creamos una función de Propagación para hacer un cálculo iterativo que nos muestre las probabilidades de Churn

In [ ]:
def propagate(G):
    next_scores = {}
    for node_id in G.nodes():
        if G.nodes[node_id]['churn']==2:
            next_scores[node_id] = G.nodes[node_id]['baj2']
        else:
            next_scores[node_id] = calculate_avg_score(G, node_id)
    for node_id in next_scores:
        G.nodes[node_id]['baj2'] = next_scores[node_id]

Ejecutamos en varios pasos

In [ ]:
n_steps = 5
for i in range(n_steps):
    propagate(E)
    print("=== After {} steps ===".format(i+1))
    print("Nodo 350002 = {}".format(E.nodes[350002]['baj2']))
    print("Nodo 350017 = {}".format(E.nodes[350017]['baj2']))
    print("Nodo 350023 = {}".format(E.nodes[350023]['baj2']))
    print("Nodo 350109 = {}".format(E.nodes[350109]['baj2']))

Podemos extender este resultado a un listado de clientes en particular

Elaborado por Luis Cajachahua bajo licencia MIT (2021)